In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load data from Excel (replace with the correct file path)
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:
# Filter relevant columns
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Rename columns for easier handling
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Convert 'Tanggal' column to datetime
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Convert 'Indeks Produksi' to numeric (coercing errors)
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Create 'bulan_tahun' column to group by month and year
data_filtered['bulan_tahun'] = data_filtered['Tanggal'].dt.to_period('M')

# Monthly Summary by 'bulan_tahun'
monthly_summary = data_filtered.groupby('bulan_tahun').agg({
    'SO': 'sum',
    # Uncomment other columns as needed
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Display monthly summary
print(monthly_summary)

In [ ]:
# Set alpha value for Double Exponential Smoothing (TES)
alpha = 0.1

# Initialize columns for TES and error metrics
monthly_summary['Single'] = np.nan
monthly_summary['Double'] = np.nan
monthly_summary['Triple'] = np.nan
monthly_summary['At'] = np.nan
monthly_summary['Bt'] = np.nan
monthly_summary['Ct'] = np.nan
monthly_summary['TES Forecast'] = np.nan
monthly_summary['Error'] = np.nan
monthly_summary['MAD'] = np.nan
monthly_summary['MSE'] = np.nan
monthly_summary['MAPE'] = np.nan

# Set initial Single, Double, At, and Bt values based on the first observation
monthly_summary.loc[0, 'Single'] = monthly_summary.loc[0, 'SO']
monthly_summary.loc[0, 'Double'] = monthly_summary.loc[0, 'SO']
monthly_summary.loc[0, 'Triple'] = monthly_summary.loc[0, 'SO']
monthly_summary.loc[0, 'At'] = (2 * monthly_summary.loc[0, 'Single']) - monthly_summary.loc[0, 'Double']
monthly_summary.loc[0, 'Bt'] = 0  # Set initial trend (Bt) to 0
monthly_summary.loc[0, 'Ct'] = 0  # Set initial trend (Ct) to 0

# Apply TES formula and calculate error metrics for each subsequent row
for i in range(1, len(monthly_summary)):
    # Single Exponential smoothing (S't)
    monthly_summary.loc[i, 'Single'] = (alpha * monthly_summary.loc[i, 'SO']) + ((1 - alpha) * monthly_summary.loc[i-1, 'Single'])
    
    # Double Exponential smoothing (S''t)
    monthly_summary.loc[i, 'Double'] = (alpha * monthly_summary.loc[i, 'Single']) + ((1 - alpha) * monthly_summary.loc[i-1, 'Double'])
    
    # Triple Exponential smoothing (S'''t)
    monthly_summary.loc[i, 'Triple'] = (alpha * monthly_summary.loc[i, 'Double']) + ((1 - alpha) * monthly_summary.loc[i-1, 'Triple'])
    
    # At (level)
    monthly_summary.loc[i, 'At'] = (3 * monthly_summary.loc[i, 'Single']) - (3 * monthly_summary.loc[i, 'Double']) + monthly_summary.loc[i, 'Triple']
    
    # Bt (trend)
    monthly_summary.loc[i, 'Bt'] = ((alpha) / (2*(1 - alpha)**2)) * ((6-5*alpha)*monthly_summary.loc[i, 'Single'] - (10-8*alpha)*monthly_summary.loc[i, 'Double'] + (4-3*alpha)*monthly_summary.loc[i, 'Triple']) 
    
    # Ct (trend)
    monthly_summary.loc[i, 'Ct'] = (alpha**2)/((1 - alpha)**2) * (monthly_summary.loc[i, 'Single'] - (2*monthly_summary.loc[i, 'Double']) + monthly_summary.loc[i, 'Triple'])
    
    # TES Forecast for the next period
    monthly_summary.loc[i, 'TES Forecast'] = monthly_summary.loc[i-1, 'At'] + monthly_summary.loc[i-1, 'Bt']
    
    # Error (actual - forecast)
    monthly_summary.loc[i, 'Error'] = monthly_summary.loc[i, 'SO'] - monthly_summary.loc[i, 'TES Forecast']
    
    # MAD (Mean Absolute Deviation)
    monthly_summary.loc[i, 'MAD'] = abs(monthly_summary.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    monthly_summary.loc[i, 'MSE'] = monthly_summary.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if monthly_summary.loc[i, 'SO'] != 0:
        monthly_summary.loc[i, 'MAPE'] = (abs(monthly_summary.loc[i, 'Error']) / monthly_summary.loc[i, 'SO']) * 100
    else:
        monthly_summary.loc[i, 'MAPE'] = np.nan

# Show the result with TES forecast and error metrics
print(monthly_summary)

In [ ]:
# Forecast for future months (12 months ahead)
months_to_forecast = 12

# Get the last month from the original data
last_month = monthly_summary['bulan_tahun'].max().to_timestamp()

# Generate dummy months starting from the next month after the last month in the original data
dummy_data = pd.DataFrame({
    'bulan_tahun': pd.date_range(start=last_month + pd.offsets.MonthBegin(1), periods=months_to_forecast, freq='M').to_period('M'),
    'SO': np.zeros(months_to_forecast)  # Set SO to 0 for future months
})

# Combine original data with the dummy future data
extended_data = pd.concat([monthly_summary[['bulan_tahun', 'SO']], dummy_data], ignore_index=True)

# Initialize columns for TES forecast in the extended data
extended_data['Single'] = np.nan
extended_data['Double'] = np.nan
extended_data['Triple'] = np.nan
extended_data['At'] = np.nan
extended_data['Bt'] = np.nan
extended_data['Ct'] = np.nan
extended_data['TES Forecast'] = np.nan
extended_data['Error'] = np.nan
extended_data['MAD'] = np.nan
extended_data['MSE'] = np.nan
extended_data['MAPE'] = np.nan

# Set initial values for the TES components based on the first row of the data
extended_data.loc[0, 'Single'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'Double'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'Triple'] = extended_data.loc[0, 'SO']
extended_data.loc[0, 'At'] = (2 * extended_data.loc[0, 'Single']) - extended_data.loc[0, 'Double']
extended_data.loc[0, 'Bt'] = 0  # Assume no initial trend
extended_data.loc[0, 'Ct'] = 0  # Assume no initial trend

# Apply TES formula to the entire dataset (historical and future periods)
for i in range(1, len(extended_data)):
    # Apply TES formula for all periods (historical and future)
    extended_data.loc[i, 'Single'] = (alpha * extended_data.loc[i, 'SO']) + ((1 - alpha) * extended_data.loc[i-1, 'Single'])
    extended_data.loc[i, 'Double'] = (alpha * extended_data.loc[i, 'Single']) + ((1 - alpha) * extended_data.loc[i-1, 'Double'])
    extended_data.loc[i, 'Triple'] = (alpha * extended_data.loc[i, 'Double']) + ((1 - alpha) * extended_data.loc[i-1, 'Triple'])
    extended_data.loc[i, 'At'] = (3 * extended_data.loc[i, 'Single']) - (3 * extended_data.loc[i, 'Double']) + extended_data.loc[i, 'Triple']
    extended_data.loc[i, 'Bt'] = ((alpha) / (2*(1 - alpha)**2)) * ((6-5*alpha)*extended_data.loc[i, 'Single'] - (10-8*alpha)*extended_data.loc[i, 'Double'] + (4-3*alpha)*extended_data.loc[i, 'Triple']) 
    extended_data.loc[i, 'Ct'] = (alpha**2)/((1 - alpha)**2) * (extended_data.loc[i, 'Single'] - (2*extended_data.loc[i, 'Double']) + extended_data.loc[i, 'Triple'])
    extended_data.loc[i, 'TES Forecast'] = extended_data.loc[i-1, 'At'] + extended_data.loc[i-1, 'Bt']
    
    # Error
    extended_data.loc[i, 'Error'] = extended_data.loc[i, 'SO'] - extended_data.loc[i, 'TES Forecast']
    
    # MAD (Mean Absolute Deviation)
    extended_data.loc[i, 'MAD'] = abs(extended_data.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    extended_data.loc[i, 'MSE'] = extended_data.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if extended_data.loc[i, 'SO'] != 0:
        extended_data.loc[i, 'MAPE'] = (abs(extended_data.loc[i, 'Error']) / extended_data.loc[i, 'SO']) * 100
    else:
        extended_data.loc[i, 'MAPE'] = np.nan

# Display the extended data with TES forecast results
print(extended_data)


In [ ]:
# Plot : Actual SO vs TES Forecast
plt.figure(figsize=(14, 6))
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['SO'], label='Actual SO', marker='o')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['TES Forecast'], label='TES Forecast', marker='x')
plt.xlabel('Month-Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs TES Forecast (Including Future Months)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot : Error Evaluation
plt.figure(figsize=(14, 8))
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['Error'], label='Error', marker='o', color='red')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['MAD'], label='MAD', marker='x', color='blue')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['MSE'], label='MSE', marker='s', color='green')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['MAPE'], label='MAPE', marker='^', color='purple')

plt.xlabel('Month-Year')
plt.ylabel('Error Metrics')
plt.xticks(rotation=45)
plt.title('Error and Evaluation Metrics (Error, MAD, MSE, MAPE)')
plt.legend()
plt.tight_layout()
plt.show()